## Setup Environment
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Project-MONAI/tutorials/blob/master/3d_registration/unpaired_lung_ct.ipynb)

In [ ]:
import random
%pip install -q "monai[nibabel, tqdm]"

In [ ]:
%pip install -q matplotlib
%matplotlib inline

## Setup imports

In [ ]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import os
import shutil
import tempfile
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.nn import MSELoss
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, Dataset, CacheDataset
from monai.losses import DiceLoss, BendingEnergyLoss
from monai.metrics import compute_meandice
from monai.networks.blocks import Warp, DVF2DDF
from monai.networks.nets import LocalNet
from monai.transforms import LoadImaged, AddChanneld, ToTensord, Compose, \
    ScaleIntensityRanged, RandAffined, Resized
from monai.utils import set_determinism, first

print_config()

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.
This allows you to save results and reuse downloads.
If not specified a temporary directory will be used.

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

## Download dataset

In [ ]:
resource = "https://zenodo.org/record/3835682/files/training.zip"

compressed_file = os.path.join(root_dir, "paired_ct_lung.zip")
data_dir = os.path.join(root_dir, "paired_ct_lung")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir)
    os.rename(os.path.join(root_dir, "training"), data_dir)

## Set dataset path
Only images at the expiration time point are used.

In [ ]:
train_iterations = 20000
train_files = []
for _ in range(train_iterations):
    fix_id, moving_id = random.sample([i for i in range(1, 19)], 2)
    train_files.append(
        {
            "fixed_image": os.path.join(data_dir, "scans/case_%03d_exp.nii.gz" % fix_id),
            "moving_image": os.path.join(data_dir, "scans/case_%03d_exp.nii.gz" % moving_id),
        }
    )

val_files = [
    {
        "fixed_image": os.path.join(data_dir, "scans/case_%03d_exp.nii.gz" % fix_id),
        "moving_image": os.path.join(data_dir, "scans/case_%03d_exp.nii.gz" % moving_id),
    }
    for fix_id, moving_id in [(19, 20), (20, 21)]
]

## Set deterministic training for reproducibility

In [ ]:
set_determinism(seed=0)

## Setup transforms for training and validation
Here we use several transforms to augment the dataset:
1. LoadImaged loads the lung CT images and labels from NIfTI format files.
2. AddChanneld as the original data doesn't have channel dim, add 1 dim to construct "channel first" shape.
5. ScaleIntensityRanged extracts intensity range [-57, 164] and scales to [0, 1].
9. RandAffined efficiently performs rotate, scale, shear, translate, etc. together based on PyTorch affine transform.
10. ToTensord converts the numpy array to PyTorch Tensor for further steps.

In [ ]:
train_transforms = Compose(
    [
        LoadImaged(
            keys=["fixed_image", "moving_image"]
        ),
        AddChanneld(
            keys=["fixed_image", "moving_image"]
        ),
        ScaleIntensityRanged(
            keys=["fixed_image", "moving_image"],
            a_min=-285, a_max=3770, b_min=0.0, b_max=1.0, clip=True,
        ),
        RandAffined(
            keys=["fixed_image", "moving_image"],
            mode=('bilinear', 'bilinear'),
            prob=1.0, spatial_size=(192, 192, 208),
            rotate_range=(0, 0, np.pi/15), scale_range=(0.1, 0.1, 0.1)
        ),
        Resized(
            keys=["fixed_image", "moving_image"],
            mode=('trilinear', 'trilinear'),
            align_corners=(True, True, None, None),
            spatial_size=(96, 96, 104)
        ),
        ToTensord(
            keys=["fixed_image", "moving_image"]
        ),
      ]
)
val_transforms = Compose(
    [
        LoadImaged(
            keys=["fixed_image", "moving_image"]
        ),
        AddChanneld(
            keys=["fixed_image", "moving_image"]
        ),
        ScaleIntensityRanged(
            keys=["fixed_image", "moving_image"],
            a_min=-285, a_max=3770, b_min=0.0, b_max=1.0,
            clip=True,
        ),
        Resized(
            keys=["fixed_image", "moving_image"],
            mode=('trilinear', 'trilinear'),
            align_corners=(True, True, None, None),
            spatial_size=(96, 96, 104)
        ),
        ToTensord(
            keys=["fixed_image", "moving_image"]
        ),
    ]
)

## Check transforms in DataLoader

In [ ]:
check_ds = Dataset(data=train_files, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=1)
check_data = first(check_loader)
fixed_image = check_data["fixed_image"][0][0]
moving_image = check_data["moving_image"][0][0]

print(f"moving_image shape: {moving_image.shape}, ",
      f"fixed_image shape: {fixed_image.shape}.")

# plot the slice [:, :, 50]
plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("moving_image")
plt.imshow(moving_image[:, :, 50], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("fixed_image")
plt.imshow(fixed_image[:, :, 50], cmap="gray")

plt.show()

## Define CacheDataset and DataLoader for training and validation

Here we use CacheDataset to accelerate training and validation process, it's 10x faster than the regular Dataset.
To achieve best performance, set cache_rate=1.0 to cache all the data, if memory is not enough, set lower value.
Users can also set cache_num instead of cache_rate, will use the minimum value of the 2 settings.
And set num_workers to enable multi-threads during caching.
If want to to try the regular Dataset, just change to use the commented code below.


In [ ]:
train_ds = CacheDataset(data=train_files, transform=train_transforms,
                        cache_rate=1.0, num_workers=4)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=4)

val_ds = CacheDataset(data=val_files, transform=val_transforms,
                      cache_rate=1.0, num_workers=0)

val_loader = DataLoader(val_ds, batch_size=1, num_workers=4)

#### Create Model, Loss, Optimizer

In [ ]:
# standard PyTorch program style: create UNet, DiceLoss and Adam optimizer
device = torch.device("cuda:0")
model = LocalNet(
        spatial_dims=3,
        in_channels=2,
        out_channels=3,
        num_channel_initial=32,
        extract_levels=[0, 1, 2, 3],
        out_activation=None,
        out_initializer="zeros").to(device)
dvf2ddf = DVF2DDF(spatial_dims=3)
warp_layer = Warp(spatial_dims=3).to(device)
image_loss = MSELoss()
regularization = BendingEnergyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)

Define a forward pass function for ddf computation and warping to avoid duplicate coding.

In [ ]:
def forward(batch_data, model, dvf2ddf):
    fixed_image = batch_data["fixed_image"].to(device)
    moving_image = batch_data["moving_image"].to(device)
    dvf = model(torch.cat((moving_image, fixed_image), dim=1))
    ddf = dvf2ddf(dvf)
    pred_image = warp_layer(moving_image, ddf)
    return ddf, pred_image

## Execute a typical PyTorch training process

In [ ]:
val_period = 10
epoch_num = 1000
val_interval = 2
best_metric = -1
best_metric_epoch = -1
loss_values = list()
metric_values = list()

loss_sum = 0

for step, batch_data in enumerate(train_loader):
    optimizer.zero_grad()

    ddf, pred_image = forward(batch_data, model, dvf2ddf)

    fixed_image = batch_data["fixed_image"].to(device)
    loss = 2 * image_loss(pred_image, fixed_image)
    loss += 4 * regularization(ddf)
    loss.backward()
    optimizer.step()

    loss_sum += loss.item()
    print(f"{step}/{len(train_ds) // train_loader.batch_size}, "
          f"train_loss: {loss.item():.4f}")
    if (step + 1) % val_period == 0:
        loss_values.append(loss_sum / val_period)
        loss_sum = 0
        model.eval()
        with torch.no_grad():
            metric_sum = 0.0
            metric_count = 0
            for val_data in val_loader:

                val_ddf, val_pred_image = forward(
                    val_data, model, dvf2ddf)

                val_fixed_image = val_data["fixed_image"].to(device)
                value = - image_loss(val_pred_image, val_fixed_image)
                metric_count += len(value)
                metric_sum += value.sum().item()
            metric = metric_sum / metric_count
            metric_values.append(metric)
            if metric > best_metric:
                best_metric = metric
                best_metric_step = step
                torch.save(model.state_dict(),
                           os.path.join(root_dir, "best_metric_model.pth"))
                print("saved new best metric model")
            print(
                f"current image similarity: {metric:.4f}\n"
                f"best image similarity: {best_metric:.4f} "
                f"at step: {best_metric_step}"
            )
        model.train()

In [ ]:
print(f"train completed, "
      f"best_metric: {best_metric:.4f}  "
      f"at step: {best_metric_step}")

#### Plot the loss and metric

In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [(i + 1) * val_period for i in range(len(loss_values))]
y = loss_values
plt.xlabel("step")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Image Similarity")
x = [(i + 1) * val_period for i in range(len(metric_values))]
y = metric_values
plt.xlabel("step")
plt.plot(x, y)
plt.show()

#### Check best model output with the input image and label

In [ ]:
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
model.eval()
with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        val_ddf, val_pred_image = forward(val_data, model, dvf2ddf)

        # plot the slice [:, :, 80]
        plt.figure("check", (18, 6))
        plt.subplot(1, 3, 1)
        plt.title(f"moving_image {i}")
        plt.imshow(val_data["moving_image"][0, 0, :, :, 50], cmap="gray")
        plt.subplot(1, 3, 2)
        plt.title(f"fixed_image {i}")
        plt.imshow(val_data["fixed_image"][0, 0, :, :, 50], cmap="gray")
        plt.subplot(1, 3, 3)
        plt.title(f"pred_image {i}")
        plt.imshow(val_pred_image[0, 0, :, :, 50], cmap="gray")
        plt.show()


## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)